##  Optimizing the Number of Agents in a Call Center Using Discrete Event Simulation

In this Jupyter Notebook, we apply Discrete Event Simulation (DES) to estimate the optimal number of agents required in a call center. The goal is to ensure that a target percentage of incoming calls are answered immediately. Any call that cannot be answered right away is treated as dropped.

The following Python libraries are used:
- [**simpy**](https://pypi.org/project/simpy/), a framework for discrete-event simulation.
- [**simpy_helpers**](https://pypi.org/project/simpy-helpers/), a convenience library that simplifies working with `simpy`. Its GitHub repository can be found [here](https://github.com/bambielli/simpy_helpers).

The three main components to `simpy_helpers` are:
- **Entities**: what flows through the system (e.g. customers, jobs, calls)
- **Resources**: where Entities go for service
    - By default, resources can form queues. If queueing is not permitted (as in our call center case), additional logic must be applied to drop entities when capacity is full.
- **Source**: generates Entities over time to enter the simulation.

A conceptual diagram of `simpy_helpers` components is:
![Conceptual Diagram](images/Conceptual.png)

In this DES, we apply `simpy` and `simpy_helpers` to the following model:
- Incoming calls are represented as Entities.
- Call center agents are modeled as a Resource.
- A Source generates calls over the course of the day.
- Calls not answered immediately (when no agent is available) are routed to a special DroppedCalls Resource to record dropped demand.

![More Specific Diagram](images/Actual-Sim.png)

In this model, calls are generated using a Poisson process (see [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution), which is a common way to represent random arrivals over time. Instead of directly sampling the number of calls per interval, we model the time between arrivals (the interarrival time) as an exponential random variable. To reflect daily fluctuations, the average interarrival time changes by hour of the day according to `MEAN_TIME_BETWEEN_CALLS_MINS_BY_HOUR`.

Each incoming call is then classified as either a good call or a bad call based on the specified percentage (`GOOD_CALL_PERCENTAGE`). Good calls have service times drawn from a normal distribution with a mean of `GOOD_CALL_MEAN_DURATION` and a standard deviation of `GOOD_CALL_STDDEV`, while bad calls have much shorter durations drawn uniformly from `BAD_CALL_DURATION_RANGE`.

Finally, the number of available agents (`number_of_agents`) can be varied across simulation runs, allowing us to explore how staffing levels affect the proportion of calls that are answered versus dropped.

**Credit**: the ideas of this work are taken directly from BUS 36109 Advanced Decision Modeling with Python, taught by Prof. [Don Eisenstein](https://www.chicagobooth.edu/faculty/directory/e/donald-d-eisenstein) at the University of Chicago Booth School of Business.

In [1]:
### -1. Installing and importing libraries, defining helper functions

In [2]:
!pip install numpy -q
!pip install simpy -q
!pip install simpy-helpers -q
!pip install matplotlib -q

In [3]:
import numpy as np
import simpy
from simpy_helpers import Entity, Resource, Source, Stats
import matplotlib.pyplot as plt

In [4]:
def get_nonnegative_num_from_normal_dist(mean, st_dev):
    """
    Sample from a normal distribution with the given mean and standard deviation,
    but ensure the returned value is nonnegative.

    Used for modeling call durations that are assumed to be normally distributed, but where negative durations are not possible.

    Reference:
    https://stackoverflow.com/questions/16312006/python-numpy-random-normal-only-positive-values
    """
    x = np.random.normal(mean, st_dev)
    return x if x >= 0 else get_nonnegative_num_from_normal_dist(mean, st_dev)

### 0. Defining parameters for the simulation

In [5]:
# Simulation parameters
SIM_LENGTH_MINS = 540  # Total simulation time: 9 hours (9am–6pm)

# Call arrival parameters (varying by hour of the day)
MEAN_TIME_BETWEEN_CALLS_MINS_BY_HOUR = {
    9: 5,   # From 9–10am, calls every 8 minutes on average
    10: 10, # From 10–11am, calls every 10 minutes on average
    11: 12,
    12: 15,
    13: 9,
    14: 8,
    15: 8,
    16: 8,
    17: 7 # From 5pm-6pm, calls every 7 minutes on average
}

# Call characteristics
GOOD_CALL_PERCENTAGE = 0.60 # 60% of calls are valid leads
BAD_CALL_PERCENTAGE = 1-GOOD_CALL_PERCENTAGE # 40% are wrong numbers / quick asks

# Good call duration (Normal distribution)
GOOD_CALL_MEAN_DURATION = 15 # Average good call duration (minutes)
GOOD_CALL_STDDEV = 5 # Standard deviation of good call duration (minutes)

# Bad call duration (Uniform distribution)
BAD_CALL_DURATION_RANGE = (0, 2)  # Duration uniformly between 0–2 minutes

# Call center parameters
number_of_agents = 2 # Number of agents (will vary in scenarios)

### 1. Defining Resource class for the simulation.

In [6]:
class CallCenter(Resource):
    """
    Resource representing the call center (group of agents) that receives incoming calls.

    Responsibilities:
    - Encapsulates agent availability via the underlying Resource queue.
    - Provides `service_time(entity)` to sample the handling time of a call
      based on its type ("Good" vs "Bad").

    Assumptions:
    - `entity.attributes["call_type"]` is either "Good" or "Bad".
    - Durations are returned in minutes.
    - Uses:
        - Good calls: Normal(mean=GOOD_CALL_MEAN_DURATION, sd=GOOD_CALL_STDDEV)
        - Bad calls:  Uniform(low=BAD_CALL_DURATION_RANGE[0],
                              high=BAD_CALL_DURATION_RANGE[1])
    """

    def service_time(self, entity):
        call_type = entity.attributes["call_type"]

        if call_type == "Good":
            return float(get_nonnegative_num_from_normal_dist(GOOD_CALL_MEAN_DURATION, GOOD_CALL_STDDEV))
        elif call_type == "Bad":
            low, high = BAD_CALL_DURATION_RANGE
            return float(np.random.uniform(low, high))
        else:
            raise ValueError(f"Unknown call_type: {call_type!r}")

### 1b. Defining a *Renege* resource for dropped calls
- Dropped calls are routed to a special DroppedCalls resource.
- This resource has effectively infinite capacity (so no queue forms).
- Each dropped call spends exactly 1 time unit there.
- The total processing time at this resource therefore equals the total number of dropped calls.
- This provides an elegant alternative to counting drops via an entity attribute.

In [7]:
class DroppedCalls(Resource):
    """
    Resource to account for dropped/reneged calls.

    Idea:
    - Give this resource very large capacity so no queue ever forms.
    - Each dropped call "uses" this resource for a deterministic time of 1
      time unit (minute), so the total processing time here equals the count
      of dropped calls.

    Usage:
    - Instantiate with a large capacity, e.g.:
        dropped_calls = DroppedCalls(env, capacity=10**9)
    """
    def service_time(self, entity):
        # Deterministic 1-minute service time = 1 'unit' per dropped call.
        return 1

### 2. Defining Entity class for the simulation

In [8]:
class Call(Entity):
    """
    A single incoming phone call passing through the system.

    Lifecycle:
    1) Arrives at the call center.
    2) If an agent is free *right now*, the call is answered immediately and
       processed at `call_center` (no waiting allowed).
    3) Otherwise, the call is dropped (reneges) and is routed to the
       `DroppedCalls` resource for a deterministic 1-minute 'accounting' stay.

    Attributes set upstream (in Source):
    - call_id (int/str): Unique identifier for the call.
    - call_type (str): Either "Good" or "Bad".
    """

    def process(self):
        # No-wait policy:
        # - If a slot is free now, proceed with normal processing.
        # - If not, mark as dropped and send to DroppedCalls for 1 time unit.
        if call_center.count < call_center.capacity:
            self.attributes["outcome"] = "answered"

            # (Optional) debug print:
            print(f"CALL ANSWERED (type: {self.attributes["call_type"]}): {call_center.count} of {call_center.capacity} agents busy")

            # Reserve an agent (simpy_helpers bookkeeping)
            yield self.wait_for_resource(call_center)

            # Be processed by the call center (duration determined by call type)
            yield self.process_at_resource(call_center)

            # Release the agent
            self.release_resource(call_center)

        else:
            self.attributes["outcome"] = "dropped"

            # (Optional) debug print:
            print(f"*** CALL DROPPED (type: {self.attributes["call_type"]}): {call_center.count} of {call_center.capacity} agents busy")

            # Enter DroppedCalls to 'account' for the drop (1 time unit)
            # Capacity is effectively infinite, so this should never block.
            yield self.wait_for_resource(dropped_calls)
            yield self.process_at_resource(dropped_calls)
            self.release_resource(dropped_calls)


### 3. Defining Source class for the simulation

In [9]:
class CallSource(Source):
    """
    Source that generates incoming phone calls.

    Responsibilities:
    - Determines interarrival times between calls, which vary by hour of day
      using the dictionary `MEAN_TIME_BETWEEN_CALLS_MINS_BY_HOUR`.
    - Creates new `Call` entities and assigns their attributes.

    Methods:
    - interarrival_time(): Returns the time until the next call, based on
      current simulation hour (drawn from an exponential distribution).
    - build_entity(): Constructs a `Call` object with a `call_type`
      ("Good" or "Bad") assigned probabilistically.

    Notes:
    - `call_type` is sampled using `GOOD_CALL_PERCENTAGE`.
    - A unique `call_id` is also assigned for tracking/logging.
    """

    def interarrival_time(self):
        # Determine current hour from simulation time
        current_hour = int(self.env.now // 60) + 9  # env.now in minutes, starting at 9am
        try:
            mean_time = MEAN_TIME_BETWEEN_CALLS_MINS_BY_HOUR[current_hour]
        except KeyError:
            raise KeyError(
                f"No mean interarrival time defined for hour {current_hour}. "
                "Please update MEAN_TIME_BETWEEN_CALLS_MINS_BY_HOUR to cover all hours "
                "in the simulation."
            )
        # Exponential distribution for interarrival times
        return np.random.exponential(mean_time)

    def build_entity(self):
        # Determine call type based on GOOD_CALL_PERCENTAGE
        if np.random.rand() < GOOD_CALL_PERCENTAGE:
            call_type = "Good"
        else:
            call_type = "Bad"

        attributes = {
            "call_type": call_type,
        }

        # Create call entity
        call = Call(self.env, attributes)
        
        return call

### 4. Run your simulation

In [10]:
# Reproducibility: fix the RNG seed so results are consistent across runs.
# Comment out this line if you want a randomized run each time instead.
np.random.seed(422)

# Create the SimPy environment (time unit: minutes)
env = simpy.Environment()

# Instantiate the call-center resource with the chosen number of agents (capacity)
call_center = CallCenter(env, capacity=number_of_agents)

# Instantiate the "renege" resource for dropped calls
# Capacity is set very large so it never blocks
dropped_calls = DroppedCalls(env, capacity=10**9)

# Instantiate the source that generates calls
call_source = CallSource(env)

# Start the source’s generator process.
# Set debug=True if you want it to print its own debug output
env.process(call_source.start(debug=True))

# Advance the simulation until the configured end time (in minutes)
env.run(until=SIM_LENGTH_MINS)

Debug is Enabled
Call 1 created_at: 1.420162081113496 attributes: {'call_type': 'Good', 'priority': 1, 'disposed': False, 'type': <class '__main__.Call'>}
CALL ANSWERED (type: Good): 0 of 2 agents busy
Call 1 requesting CallCenter: 1.420162081113496
Call 1 started processing at CallCenter : 1.420162081113496
Call 2 created_at: 2.3878626930828846 attributes: {'call_type': 'Bad', 'priority': 1, 'disposed': False, 'type': <class '__main__.Call'>}
CALL ANSWERED (type: Bad): 1 of 2 agents busy
Call 2 requesting CallCenter: 2.3878626930828846
Call 2 started processing at CallCenter : 2.3878626930828846
Call 2 finished at CallCenter: 3.9400936127415247
Call 2 disposed: 3.9400936127415247
Call 3 created_at: 12.88751548836839 attributes: {'call_type': 'Bad', 'priority': 1, 'disposed': False, 'type': <class '__main__.Call'>}
CALL ANSWERED (type: Bad): 1 of 2 agents busy
Call 3 requesting CallCenter: 12.88751548836839
Call 3 started processing at CallCenter : 12.88751548836839
Call 3 finished at 